In [6]:
import numpy as np
import pandas as pd
import itertools
import nltk
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import sys
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [15]:
avis_df = pd.read_csv('data_preprocess_sans_stopwords.csv')
details_df = pd.read_csv('details.csv')
infos_scrapping_df = pd.read_csv('infos_scrapping.csv')
jeux_df = pd.read_csv('jeux.csv')

In [ ]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(bow, y_binary, test_size=0.2, random_state=42)

# Initialize the CountVectorizer to convert text into numerical features
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [16]:
cv = CountVectorizer()
avis_df['comment'] = avis_df['comment'].fillna('')
bow = cv.fit_transform(avis_df['comment'])

In [17]:
y = avis_df["note"]

# transform the note column into a binary classification target variable
y_binary = np.where(y > 7, 1, -1)

In [18]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(bow, y_binary, test_size=0.2, random_state=42)

In [19]:
# Find the best value of K
k_values = np.arange(1, 500, 2)  # Range of K values to try
accuracy_scores = []

In [20]:
for k in k_values:
    # Train the K-Nearest Neighbors classifier
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    print(k , flush = true)
    # Predict the labels of the test set
    y_pred = knn.predict(X_test)
    
    # Calculate the accuracy of the classifier
    accuracy = np.mean(y_pred == y_test)
    accuracy_scores.append(accuracy)

KeyboardInterrupt: 

In [ ]:
# Find the best K based on the accuracy scores
best_k = k_values[np.argmax(accuracy_scores)]

In [ ]:
# Plot the accuracy scores
plt.plot(k_values, accuracy_scores)
plt.xlabel('K')
plt.ylabel('Accuracy')
plt.title('Accuracy vs. K')
plt.show()

In [ ]:
# Train the K-Nearest Neighbors classifier with the best K
best_knn = KNeighborsClassifier(n_neighbors=best_k)
best_knn.fit(X_train, y_train)

In [ ]:
# Plot the decision boundary
# Note: This code assumes that the data is 2-dimensional for visualization purposes
if X_train.shape[1] == 2:
    h = 0.02  # Step size in the mesh
    x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
    y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = best_knn.predict(np.c_[xx.ravel(), yy.ravel()])

    # Plot the decision boundary
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, alpha=0.8)
    plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=plt.cm.Paired)
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.title('Decision Boundary')
    plt.show()

In [ ]:
# Predict the labels of the test set using the best K
y_pred = best_knn.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = np.mean(y_pred == y_test)
print("Accuracy:", accuracy)